In [321]:
## automation exercise
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from statsmodels.stats.power import TTestIndPower
from scipy.stats import ttest_ind

In [323]:
data = pd.read_csv('disable vm click (12).csv') ## needs to have experiment_group, os_platform, some success metric
assignment = pd.read_csv('disable vm click (13).csv') ## a set sql query

In [324]:
data.head()

,EXPERIMENT_GROUP,DS,OS_PLATFORM,SUBSCRIPTION_BEFORE_PURCHASE,SOURCE,DURATION,PROFILES_PURCHASED_CT,REVENUE
0,control,2022-11-08,Android,free,viewedMe_blurredProfile,3 month,1,98.170918
1,control,2022-10-28,Android,free,offer,12 month,1,172.990000
2,click_disabled,2022-11-11,Android,xtra,inbox_messages_unsend,1 month,2,68.021731
3,click_disabled,2022-11-09,Android,free,inbox_messages_expiringPhotos,1 month,5,175.529333
4,control,2022-11-03,Android,free,profileDrawer_incognito,12 month,1,172.990000


In [325]:
# standardize column names

def set_columns(df, kpi, population='', platform='OS_PLATFORM', exp_group = 'EXPERIMENT_GROUP'):
    df = df.rename(columns={kpi: 'SUCCESS', platform: 'OS_PLATFORM', exp_group: 'EXPERIMENT_GROUP'})
    if population == '':
        return df 
    df = df.rename(columns={population: 'ELIGIBLE_PROFILES'})
    return df

In [326]:
# ready tbl for ttest; merge results with eligible profiles table if applicable

def group_df(df, dimension=[], assign_df=pd.DataFrame()):
    group_by = ['EXPERIMENT_GROUP', 'OS_PLATFORM'] + dimension
    df = df.groupby(group_by).sum().reset_index()
    if len(assign_df.columns) > 0:
        df = df.merge(assign_df, on=['EXPERIMENT_GROUP', 'OS_PLATFORM'])
        df = df.rename(columns={'COUNT(DISTINCT PROFILE_ID)': 'ELIGIBLE_PROFILES'})
    df['CONVERSION'] = df['SUCCESS']/df['ELIGIBLE_PROFILES']
    if 'REVENUE' in df.columns:
        df['ARPU'] = df['REVENUE']/df['ELIGIBLE_PROFILES']
    return df

In [327]:
# # function to group results by dimension and merge results df and eligible profiles df
# def group_df(df, assign_df, dimension=[]):
    
#     group_by = ['EXPERIMENT_GROUP', 'OS_PLATFORM'] + dimension
#     df = df.groupby(group_by).sum().reset_index()
#     df = df.merge(assign_df, on=['EXPERIMENT_GROUP', 'OS_PLATFORM'])
#     df = df.rename(columns={'COUNT(DISTINCT PROFILE_ID)': 'ELIGIBLE_PROFILES'})
#     return df

In [438]:
# (helper function) find p value for conversion - split by android and ios

def find_p_value(df, dimension=''):
    os_platforms = np.unique(df['OS_PLATFORM'])
    for os_platform in os_platforms:
        os_df = df[df['OS_PLATFORM'] == os_platform].reset_index()
        os_df.to_excel(writer, sheet_name=f'{str(os_platform)[:3]}_{dimension}_con', index=False)
        
        a_dist = np.zeros(os_df['ELIGIBLE_PROFILES'][0])
        a_dist[:os_df['SUCCESS'][0]] = 1
        b_dist = np.zeros(os_df['ELIGIBLE_PROFILES'][1])
        b_dist[:os_df['SUCCESS'][1]] = 1
        
        z_stat, pval = ttest_ind(a_dist, b_dist, equal_var=True)
        print(f'{os_platform} {dimension} z statistic: {z_stat:.2f}')
        print(f'{os_platform} {dimension} p-value: {pval:.3f}')

In [441]:
# (helper function) find p value for arpu - split by android and ios

def find_arpu_p_value(df, dimension=''):
    os_platforms = np.unique(df['OS_PLATFORM'])
    for os_platform in os_platforms:
        os_df = df[df['OS_PLATFORM'] == os_platform].reset_index()
        os_df.to_excel(writer, sheet_name=f'{str(os_platform)[:3]}_{dimension}_arpu', index=False)
        
        arppu_a = os_df['REVENUE'][0]/os_df['SUCCESS'][0]
        arppu_b = os_df['REVENUE'][1]/os_df['SUCCESS'][1]
        a_dist = np.zeros(os_df['ELIGIBLE_PROFILES'][0])
        a_dist[:os_df['SUCCESS'][0]] = arppu_a
        b_dist = np.zeros(os_df['ELIGIBLE_PROFILES'][1])
        b_dist[:os_df['SUCCESS'][1]] = arppu_b
        
        z_stat, pval = ttest_ind(a_dist, b_dist, equal_var=True)
        print(f'{os_platform} {dimension} z statistic: {z_stat:.2f}')
        print(f'{os_platform} {dimension} p-value: {pval:.3f}')

In [330]:
# (helper function) find p values if cutting by dimension

def dimension_p_value(df, dimension):
    dim_df = df.rename(columns={dimension: 'key'})
    dim_df.set_index(keys=['key'], inplace=True)
    values = np.unique(dim_df.index)
    for value in values:
        d_df = dim_df.loc[(dim_df.index == value)]
        find_p_value(d_df, value)
    return d_df

In [331]:
# (helper function) find p values if cutting by dimension

def dimension_arpu_p_value(df, dimension):
    dim_df = df.rename(columns={dimension: 'key'})
    dim_df.set_index(keys=['key'], inplace=True)
    values = np.unique(dim_df.index)
    for value in values:
        d_df = dim_df.loc[(dim_df.index == value)]
        find_arpu_p_value(d_df, value)
    return d_df

In [332]:
# find p values for conversion given set of dimensions - split by android and ios
## dimensions: any cuts to the data (excluding os_platform which is already included)
## assign_df: assignment dataframe (if applicable)

def find_p_values(df, dimensions=[], assign_df=pd.DataFrame()):
    overall_df = group_df(df, [], assign_df)
    find_p_value(overall_df, 'overall')
    if dimensions == []:
        return
    for arg in dimensions:
        dimension = []
        dimension.append(arg)
        dim_df = group_df(df, dimension, assign_df)
        dimension_p_value(dim_df, arg)

In [333]:
# find p values for arpu given set of dimensions - split by android and ios
## dimensions: any cuts to the data (excluding os_platform which is already included)
## assign_df: assignment dataframe (if applicable)

def find_arpu_p_values(df, dimensions=[], assign_df=pd.DataFrame()):
    overall_df = group_df(df, [], assign_df)
    find_arpu_p_value(overall_df, 'overall')
    if dimensions == []:
        return
    for arg in dimensions:
        dimension = []
        dimension.append(arg)
        dim_df = group_df(df, dimension, assign_df)
        dimension_arpu_p_value(dim_df, arg)

In [416]:
writer = pd.ExcelWriter('test2.xlsx', engine='xlsxwriter')

In [413]:
df2 = pd.read_csv('holiday campaigns (6).csv')

In [414]:
df2 = set_columns(df2, 'PURCHASES', 'UNIQUE_IMPRESSIONS', 'PLATFORM', 'VARIANT')

In [384]:
df2.head()

,CAMPAIGN_NAME,MESSAGE_VARIATION_ID,AUDIENCE_TYPE,OS_PLATFORM,EXPERIMENT_GROUP,ELIGIBLE_PROFILES,UNIQUE_CLICKS,SUCCESS,UNIQUE_PURCHASES,REVENUE
0,Revenue_PastPurchased_Xtra_Android_Inbox_11.22...,bfde8f1a-7d2b-4bd4-94f0-84806d4585e8,PastPurchased,android,discount,3672,1894,93,92,1151.070000
1,Revenue_PastPurchased_Xtra_iOS_Inbox_11.22.2022,d6728f54-38f0-4175-b0d0-0ab4626ceadb,PastPurchased,ios,full_price,7289,7196,77,77,2619.230000
2,Revenue_PastPurchased_Xtra_Android_Inbox_11.22...,a7673ce1-346a-461c-a3a1-7955e6597f57,PastPurchased,android,full_price,3550,1684,38,35,1150.800000
3,Revenue_PastPurchased_Xtra_iOS_Inbox_11.22.2022,c106c73d-987c-4470-b6b3-33f24b7a2dbe,PastPurchased,ios,discount,7219,7148,230,230,2944.640147
4,Revenue_NeverPurchased_Xtra_Android_Inbox_11.2...,b618101d-1c12-45be-8bb3-e2e378f55c4c,NeverPurchased,android,full_price,29253,14444,113,109,3060.035935


In [424]:
group_df(df2, ['AUDIENCE_TYPE'])

,EXPERIMENT_GROUP,OS_PLATFORM,AUDIENCE_TYPE,ELIGIBLE_PROFILES,UNIQUE_CLICKS,SUCCESS,UNIQUE_PURCHASES,REVENUE,CONVERSION,ARPU
0,discount,android,NeverPurchased,28818,14719,181,179,1886.103249,0.006281,0.065449
1,discount,android,PastPurchased,3672,1894,93,92,1151.070000,0.025327,0.313472
2,discount,ios,NeverPurchased,53843,53179,446,441,5353.544869,0.008283,0.099429
3,discount,ios,PastPurchased,7219,7148,230,230,2944.640147,0.031860,0.407901
4,full_price,android,NeverPurchased,29253,14444,113,109,3060.035935,0.003863,0.104606
5,full_price,android,PastPurchased,3550,1684,38,35,1150.800000,0.010704,0.324169
6,full_price,ios,NeverPurchased,53966,53270,210,209,5553.887670,0.003891,0.102915
7,full_price,ios,PastPurchased,7289,7196,77,77,2619.230000,0.010564,0.359340


In [439]:
find_arpu_p_values(df2, ['AUDIENCE_TYPE'])

android overall z statistic: -2.94
android overall p-value: 0.003
ios overall z statistic: 0.26
ios overall p-value: 0.793
android NeverPurchased z statistic: -3.56
android NeverPurchased p-value: 0.000
ios NeverPurchased z statistic: -0.41
ios NeverPurchased p-value: 0.682
android PastPurchased z statistic: -0.18
android PastPurchased p-value: 0.861
ios PastPurchased z statistic: 1.00
ios PastPurchased p-value: 0.318


In [440]:
find_p_values(df2, ['AUDIENCE_TYPE'])

android overall z statistic: 6.09
android overall p-value: 0.000
ios overall z statistic: 12.64
ios overall p-value: 0.000
android NeverPurchased z statistic: 4.11
android NeverPurchased p-value: 0.000
ios NeverPurchased z statistic: 9.28
ios NeverPurchased p-value: 0.000
android PastPurchased z statistic: 4.66
android PastPurchased p-value: 0.000
ios PastPurchased z statistic: 8.94
ios PastPurchased p-value: 0.000


In [427]:
writer.save()